In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def create_scan_mask(image, area_threshold=10000):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask_clean = np.zeros_like(gray)
    for contour in contours:
        if cv2.contourArea(contour) > area_threshold:
            cv2.drawContours(mask_clean, [contour], -1, 255, thickness=cv2.FILLED)
    return mask_clean



In [7]:

dataset_path = "/content/drive/MyDrive/OTU-2D-Dataset-main/OTU-2D-Dataset-main/dataset_split/train/images"
output_path = "/content/drive/MyDrive/OTU-2D-Dataset-main/OTU-2D-Dataset-main/dataset_split/train/scan_mask"
os.makedirs(output_path, exist_ok=True)

valid_image_files = []
for f in os.listdir(dataset_path):
    if f.lower().endswith(('.jpg', '.jpeg', '.png')):
        path = os.path.join(dataset_path, f)
        img = cv2.imread(path)
        if img is not None:
            valid_image_files.append(f)

valid_image_files = sorted(valid_image_files)

for image_name in tqdm(valid_image_files, desc="Tạo scan mask"):
    image_path = os.path.join(dataset_path, image_name)
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image_rgb.shape[:2]
    scan_mask = create_scan_mask(image_rgb)
    if scan_mask.shape != (h, w):
        scan_mask = cv2.resize(scan_mask, (w, h), interpolation=cv2.INTER_NEAREST)
    base_name = os.path.splitext(image_name)[0]
    scan_mask_name = f"{base_name}_scan_mask.png"
    scan_mask_path = os.path.join(output_path, scan_mask_name)
    cv2.imwrite(scan_mask_path, scan_mask)

Tạo scan mask: 100%|██████████| 1177/1177 [00:20<00:00, 56.12it/s]
